In [11]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.TwoDoF as TwoDoF

from matplotlib.lines import Line2D

me.mechanics_printing()

In [12]:
half = sym.Rational(1,2)
fwt_panels = 1
p = TwoDoF.base_params(panels = fwt_panels,dofs = 1)

In [13]:
#Define refernce frames
wing_root_frame = mf.HomogenousTransform().R_x(p.q[0])

# Define Elements
M_innerWing = ele.RigidElement.PointMass(wing_root_frame,p.m_w,True)
M_Wingtip = ele.RigidElement.PointMass(wing_root_frame.Translate(0,p.l_com,0),p.m,True)
M_WingtipWeight = ele.RigidElement.PointMass(wing_root_frame.Translate(0,p.l_m,0),p.m_1,True)

# Define Aero Forces
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,wing_root_frame.Translate(0,p.y_i,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = p.alpha_1,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = p.c_d_max,
                               linear = False)

# split Main Wing into segments
forces = []
for i in range(fwt_panels):
    seg_width = p.s/fwt_panels
    yi = seg_width/2 + i*seg_width
    forces.append(wing_AeroForces.subs({p.y_i:yi,p.a_0:p.a[i]})*seg_width)

Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
wing_AeroForces = ef.ExternalForce(Q)

In [14]:
# create instance of the model
sm = mf.SymbolicModel.FromElementsAndForces(p,[M_innerWing,M_Wingtip,M_WingtipWeight],ExtForces = wing_AeroForces)

In [15]:
sm.to_file('1Dof_Model.py')